In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import requests
from collections import Counter

In [2]:
data_dir = r'C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\\'

In [3]:
df_images = pd.read_csv(data_dir + 'images.csv')
df_reactions = pd.read_csv(data_dir + 'reactions.csv')
df_user_interactions = pd.read_csv(data_dir + 'user_interactions.csv')
df_users = pd.read_csv(data_dir + 'users.csv')

In [4]:
df_users

,id,chat_id,created_at
0,1,250465332,12.10.2022 8:26:07
1,3,723941656,12.10.2022 18:23:14
2,5,192193316,12.10.2022 18:39:57
3,8,213316727,13.10.2022 19:11:52
4,166,559843492,14.10.2022 22:36:18
5,175,192193316,NaN
6,176,192193316,NaN
7,177,192193316,NaN
8,178,192193316,NaN
9,179,192193316,NaN


In [5]:
df_reactions

,id,comment
0,0,no toxic
1,1,toxic other
2,2,repeat
3,3,not mem
4,4,violence
5,5,xenophobia
6,6,drugs
7,7,misogyny
8,8,suicide
9,9,porn


In [6]:
df_images

,id,image_url,uri,topic,nick,created_at
0,1,https://s00.yaplakal.com/pics/pics_original/9/...,7664869.jpg,topic1369795.html,Невидимый,11.10.2022 18:26:02
1,2,https://s00.yaplakal.com/pics/pics_original/0/...,7664870.jpg,topic1369795.html,Невидимый,11.10.2022 18:26:02
2,3,https://s00.yaplakal.com/pics/pics_original/1/...,7664871.jpg,topic1369795.html,Невидимый,11.10.2022 18:26:02
3,4,https://s00.yaplakal.com/pics/pics_original/2/...,7664872.jpg,topic1369795.html,Невидимый,11.10.2022 18:26:02
4,5,https://s00.yaplakal.com/pics/pics_original/4/...,7664874.jpg,topic1369795.html,Невидимый,11.10.2022 18:26:02
...,...,...,...,...,...,...
18410,18411,https://s00.yaplakal.com/pics/pics_original/8/...,17121028.jpg,topic2494436.html,roaroma,11.10.2022 18:26:02
18411,18412,https://s00.yaplakal.com/pics/pics_original/7/...,17122127.jpg,topic2494436.html,Понтифик,11.10.2022 18:26:02
18412,18413,https://s00.yaplakal.com/pics/pics_original/4/...,17122134.jpg,topic2494436.html,Понтифик,11.10.2022 18:26:02
18413,18414,https://s00.yaplakal.com/pics/pics_original/8/...,17122148.jpg,topic2494436.html,Понтифик,11.10.2022 18:26:02


In [7]:
df_user_interactions

,id,image_id,user_id,react,created_at
0,5,7146,250465332,3,12.10.2022 8:26:07
1,6,6505,250465332,3,12.10.2022 8:26:18
2,7,13951,250465332,0,12.10.2022 8:26:38
3,8,3789,250465332,1,12.10.2022 8:26:44
4,9,17653,250465332,0,12.10.2022 8:26:50
...,...,...,...,...,...
848,861,12445,192193316,1,NaN
849,862,16539,326525586,6,NaN
850,863,14459,326525586,0,NaN
851,864,14263,326525586,0,NaN


In [8]:
df_user_interactions_merged = df_user_interactions.merge(df_images, left_on='image_id', right_on='id')
df_user_interactions_merged

,id_x,image_id,user_id,react,created_at_x,id_y,image_url,uri,topic,nick,created_at_y
0,5,7146,250465332,3,12.10.2022 8:26:07,7146,https://s00.yaplakal.com/pics/pics_original/7/...,10910387.jpg,topic1725920.html,Невидимый,11.10.2022 18:26:02
1,6,6505,250465332,3,12.10.2022 8:26:18,6505,https://s00.yaplakal.com/pics/pics_original/9/...,10747369.jpg,topic1709412.html,Точегонет,11.10.2022 18:26:02
2,7,13951,250465332,0,12.10.2022 8:26:38,13951,https://s00.yaplakal.com/pics/pics_original/6/...,13996726.jpg,topic2072826.html,Невидимый,11.10.2022 18:26:02
3,8,3789,250465332,1,12.10.2022 8:26:44,3789,https://s00.yaplakal.com/pics/pics_original/6/...,9987066.jpg,topic1626577.html,zolotnik,11.10.2022 18:26:02
4,9,17653,250465332,0,12.10.2022 8:26:50,17653,https://s00.yaplakal.com/pics/pics_original/0/...,16702250.jpg,topic2436308.html,KKS137,11.10.2022 18:26:02
...,...,...,...,...,...,...,...,...,...,...,...
847,861,12445,192193316,1,NaN,12445,https://s00.yaplakal.com/pics/pics_original/7/...,12954537.jpg,topic1940574.html,Невидимый,11.10.2022 18:26:02
848,862,16539,326525586,6,NaN,16539,https://s00.yaplakal.com/pics/pics_original/8/...,16180208.jpg,topic2357606.html,gizmo1251,11.10.2022 18:26:02
849,863,14459,326525586,0,NaN,14459,https://s00.yaplakal.com/pics/pics_original/3/...,15244043.jpg,topic2230226.html,ХэнкМуди,11.10.2022 18:26:02
850,864,14263,326525586,0,NaN,14263,https://s00.yaplakal.com/pics/pics_original/2/...,14993032.jpg,topic2199890.html,Невидимый,11.10.2022 18:26:02


In [9]:
len(df_user_interactions_merged)

852

In [11]:
#Сохраняем картинки
for row in df_user_interactions_merged.values:
    url = row[6]
    image_name = url[url.rindex('/') + 1 :]
    r = requests.get(url, stream=True)
    with open(data_dir + 'img/' + image_name, 'wb') as f:
        for chunk in r.iter_content():
            f.write(chunk)

In [12]:
train, test = train_test_split(df_user_interactions_merged, test_size=0.25, random_state=42, shuffle=False)

In [15]:
test_dict = {}
for row in test.values:
    dictRow = {}
    dictRow["label"] = row[3]

    url = row[6]
    image_name = url[url.rindex('/') + 1 :]
    dictRow["img"] = 'img/' + image_name
    
    test_dict[image_name] = dictRow

with open(data_dir + 'test.jsonl', "w") as rf:
    for k in test_dict:
        rf.write(json.dumps(test_dict[k]) + "\n")
        
len(test_dict)

211

In [16]:
train_dict = {}
for row in train.values:
    dictRow = {}
    dictRow["label"] = row[3]

    url = row[6]
    image_name = url[url.rindex('/') + 1 :]
    dictRow["img"] = 'img/' + image_name
    
    train_dict[image_name] = dictRow

with open(data_dir + 'train.jsonl', "w") as rf:
    for k in train_dict:
        rf.write(json.dumps(train_dict[k]) + "\n")
        
len(train_dict)

631

In [17]:
c_test = Counter(test['react']) 
c_train = Counter(train['react']) 

sum_test = sum([c_test[k] for k in c_test])
sum_train = sum([c_train[k] for k in c_train])

[(k, c_test[k] / sum_test, c_train[k] / sum_train) for k in c_test]

[(0, 0.3755868544600939, 0.3333333333333333),
 (6, 0.028169014084507043, 0.003129890453834116),
 (1, 0.28169014084507044, 0.21283255086071987),
 (5, 0.09389671361502347, 0.06259780907668232),
 (7, 0.018779342723004695, 0.03129890453834116),
 (4, 0.09389671361502347, 0.054773082942097026),
 (9, 0.07981220657276995, 0.04225352112676056),
 (3, 0.018779342723004695, 0.24569640062597808),
 (2, 0.004694835680751174, 0.001564945226917058),
 (10, 0.004694835680751174, 0.0)]

In [18]:
len(train_dict) + len(test_dict)

842